In [3]:
# Basis-Setup für Datenanalyse und Embedding
import json
import pandas as pd
import numpy as np
import re
from typing import List, Dict
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import warnings
import tiktoken
import openai
from dotenv import load_dotenv
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
# Import der Funktionen aus text_to_dqr.py
from text_to_dqr import text_to_dqr, system_prompt
from utils import count_tokens


warnings.filterwarnings("ignore")

/Users/hxx/Library/CloudStorage/OneDrive-Persönlich/Dokumente/Master/2. Semester/Web Mining/web_mining_lm/pythonProject1/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/hxx/Library/CloudStorage/OneDrive-Persönlich/Dokumente/Master/2. Semester/Web Mining/web_mining_lm/pythonProject1/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

print("OpenAI API bereit")

OpenAI API bereit


In [5]:
def load_json(path='testdata/filtered_minister_with_content.json'):
    """Lädt JSON-Daten der Politiker-Biografien"""
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    print(f"Anzahl geladener Politiker: {len(data)}")
    return data

# Beispiel ausführen
data = load_json()

Anzahl geladener Politiker: 165


In [6]:
print(system_prompt)


    Du bist ein Experte für die deutsche Bildungsordnung. Du bekommst kurze biographische Texte von Politikern. Diese sind aus Wikipedia-Artikeln extrahiert.
    Deine Aufgabe ist es, aus dem Text den höchsten erreichten Bildungsabschluss nach dem Deutschen Qualifikationsrahmen (DQR) zu erkennen und zurückzugeben.
    
    AUSGABEFORMAT (streng!):
    - Entweder exakt: DQR-Niveau; Sehr kurzer Kommentar zum höchsten Bildungsabschluss; Confidence
      Beispiele:
      8; Dr. der Wirtschaftspsychologie; 5
      7; Volljurist mit 2. Staatsexamen; 4
    - ODER (nur wenn keinerlei bildungsrelevante Information in ALLEN Abschnitten vorhanden ist):
      0; Keine Angabe zum Bildungsabschluss (vermutlich falsches Retrieval); -
    
    Hinweis zum Kommentar:
    - Der Kommentar bleibt sehr kurz. Für 0-Fälle gib einen knappen Grund an (z. B. „vermutlich falsches Retrieval“ oder „explizit: ohne Abschluss/Abbruch“).
    
    Confidence-Score (Definition):
    C=3 (sehr sicher): Explizite Nennung

In [7]:
def extract_content_sections(person):
    """Extrahiert alle Content-Abschnitte von einem Politiker"""
    all_sections = []  

    for content in person.get('neo4j_content', []):
        all_sections.append({
            'politician_name': f"{person['Vorname']} {person['Nachname']}",
            'content_id': content['content_id'],
            'section_content': content['section_content']
        })

    print(f"Gesamtanzahl Abschnitte: {len(all_sections)}")
    return all_sections

# Beispiel ausführen
person = next(item for item in data if item["ID"] == "#008")

example_sections = extract_content_sections(person)
print(example_sections)

Gesamtanzahl Abschnitte: 7
[{'politician_name': 'Willy Brandt', 'content_id': 'ffc627a3ca460e74f90fdd8114de2841645549ce', 'section_content': 'Monographien[Bearbeiten | Quelltext bearbeiten]\nKrieg in Norwegen. 9. April – 9. Juni 1940 (schwedischer Originaltitel: Kriget i Norge), übersetzt von Benedict Christ, Basel. Europa Verlag, Zürich / New York 1942. Aktualisierte Auflage: Europa Verlag, Zürich 2010, ISBN 978-3-905811-23-0. Zur Nachkriegspolitik der deutschen Sozialisten. Jocke Leufvmark. Stockholm 1944, DNB 992975301. (Gemeinsam mit August Enderle, Irmgard Enderle, Stefan Szende und Ernst Behm). Forbrytere og andre tyskere („Verbrecher und andere Deutsche“), Oslo 1946; erste dt. Ausgabe: Verbrecher und andere Deutsche. Ein Bericht aus Deutschland 1946. Bearb. u. hrsg. von Einhart Lorenz. J.H.W. Dietz Nachf., Bonn 2007, ISBN 978-3-8012-0380-1. (Willy-Brandt-Dokumente, Band 1).[107] Mein Weg nach Berlin. aufgezeichnet von Léo Lania. Kindler-Verlag, München 1960, DNB 450589072; siehe

In [8]:
def chunk_content_sections(sections, max_tokens=1000, overlap_tokens=200):
    """Chunkt Content-Sections die zu lang sind mit Overlap"""
    chunked_sections = []
    
    for section in sections:
        content = section['section_content']
        token_count = count_tokens(content)
        
        if token_count <= max_tokens:
            # Section ist kurz genug - einfach übernehmen
            chunked_sections.append(section)
        else:
            # Section ist zu lang - aufteilen
            words = content.split()
            current_chunk = []
            current_tokens = 0
            chunk_counter = 0
            
            for word in words:
                word_tokens = count_tokens(word + " ")
                current_chunk.append(word)
                current_tokens += word_tokens
                
                if current_tokens >= max_tokens:
                    # Chunk ist voll - speichern
                    chunk_text = " ".join(current_chunk)
                    chunked_sections.append({
                        'politician_name': section['politician_name'],
                        'content_id': f"#{chunk_counter:02d}_{section['content_id']}",
                        'section_content': chunk_text
                    })
                    
                    # Overlap für nächsten Chunk vorbereiten
                    overlap_words = []
                    overlap_tokens_count = 0
                    for word in reversed(current_chunk):
                        word_tokens = count_tokens(word + " ")
                        if overlap_tokens_count + word_tokens <= overlap_tokens:
                            overlap_words.insert(0, word)
                            overlap_tokens_count += word_tokens
                        else:
                            break
                    
                    current_chunk = overlap_words
                    current_tokens = overlap_tokens_count
                    chunk_counter += 1
            
            # Letzten Chunk hinzufügen falls noch Inhalt vorhanden
            if current_chunk:
                chunk_text = " ".join(current_chunk)
                chunked_sections.append({
                    'politician_name': section['politician_name'],
                    'content_id': f"#{chunk_counter:02d}_{section['content_id']}",
                    'section_content': chunk_text
                })
    
    return chunked_sections

# Nach extract_content_sections
chunked_sections = chunk_content_sections(example_sections)
print(f"Original: {len(example_sections)} Sections")
print(f"Nach Chunking: {len(chunked_sections)} Sections")
print(chunked_sections)

Original: 7 Sections
Nach Chunking: 40 Sections
[{'politician_name': 'Willy Brandt', 'content_id': '#00_ffc627a3ca460e74f90fdd8114de2841645549ce', 'section_content': 'Monographien[Bearbeiten | Quelltext bearbeiten] Krieg in Norwegen. 9. April – 9. Juni 1940 (schwedischer Originaltitel: Kriget i Norge), übersetzt von Benedict Christ, Basel. Europa Verlag, Zürich / New York 1942. Aktualisierte Auflage: Europa Verlag, Zürich 2010, ISBN 978-3-905811-23-0. Zur Nachkriegspolitik der deutschen Sozialisten. Jocke Leufvmark. Stockholm 1944, DNB 992975301. (Gemeinsam mit August Enderle, Irmgard Enderle, Stefan Szende und Ernst Behm). Forbrytere og andre tyskere („Verbrecher und andere Deutsche“), Oslo 1946; erste dt. Ausgabe: Verbrecher und andere Deutsche. Ein Bericht aus Deutschland 1946. Bearb. u. hrsg. von Einhart Lorenz. J.H.W. Dietz Nachf., Bonn 2007, ISBN 978-3-8012-0380-1. (Willy-Brandt-Dokumente, Band 1).[107] Mein Weg nach Berlin. aufgezeichnet von Léo Lania. Kindler-Verlag, München 19

In [9]:
def embed_sections(sections, batch_size=200):
    """Erstellt Embeddings für Abschnittstexte mit OpenAI v1 API"""
    
    texts = [section['section_content'] for section in sections]
    embeddings = []
    
    # Batch-weise verarbeiten wegen Rate Limits
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        
        response = openai.embeddings.create(
            model="text-embedding-3-large",
            input=batch_texts
        )
        
        batch_embeddings = [data.embedding for data in response.data]
        embeddings.extend(batch_embeddings)
    
    return np.array(embeddings)

In [10]:
section_embeddings = embed_sections(chunked_sections)

In [11]:
education_queries = [
    "Hat die Person studiert? Welche Schule hat sie besucht? Was ist über ihren Bildungsweg bekannt? Welchen Beruf hat die Person vorher ausgeübt? Hat die Person eine Ausbildung gemacht?"
]

# Queries einbetten (kein model Parameter mehr nötig)
query_embedding = embed_sections([{'section_content': q} for q in education_queries])

In [12]:
def find_top_k_sections(query_embedding, section_embeddings, all_sections, top_k=5):
    """Findet die ähnlichsten Content-Abschnitte für eine Query"""
    # Query-Embedding ist jetzt 2D, nehmen den ersten (und einzigen) Query
    similarities = cosine_similarity(query_embedding[0:1], section_embeddings)[0]
    
    # Top-k Indizes finden
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    results = []
    for idx in top_indices:
        results.append({
            'section': all_sections[idx],
            'similarity': similarities[idx],
            'content_preview': all_sections[idx]['section_content'][:300] + "..."
        })
    
    return results

In [13]:
# Beispiel-Query: Bildungsweg
top_sections = find_top_k_sections(query_embedding, section_embeddings, chunked_sections, top_k=100)

# In DataFrame umwandeln für strukturierte Darstellung
df = pd.DataFrame([{
    "Rang": i+1,
    "Politiker": result['section']['politician_name'],
    "Score": f"{result['similarity']:.3f}",
    "token_count": count_tokens(result['section']['section_content']),
    "Inhalt": result['section']['section_content']
} for i, result in enumerate(top_sections)])

# Lange Spalteninhalte anzeigen
pd.set_option('display.max_colwidth', None)

# Anzeige
display(df)

,Rang,Politiker,Score,token_count,Inhalt
0,1,Willy Brandt,0.394,626,"zu dem fünfjährigen Herbert auf. Er nahm ihn in seinen Haushalt auf und behielt ihn während der 1920er Jahre in seiner Obhut, auch nachdem er nach dem Tod Wilhelmines eine neue Ehe mit Dorothea Stahlmann, der von Brandt wenig geliebten „Tante Dora“, eingegangen war. Herbert Frahm nannte seinen Stiefgroßvater „Papa“. In seinem Abiturzeugnis wurde Ludwig Frahm als Vater genannt.[6] Im September 1927 heiratete Herberts Mutter den Polier Emil Kuhlmann, im Februar 1928 kam Herberts Halbbruder Günther Kuhlmann zur Welt. Seit dieser Zeit sah Herbert seine Mutter „nur noch sporadisch“.[4] Rückblickend bezeichnete Brandt seine Jugend als „unbehaust“[3] und beschrieb seine Familienverhältnisse als chaotisch.[7] Brandts uneheliche Geburt, die von Zeitgenossen vielfach als Makel angesehen wurde, benutzten politische Gegner noch in der Zeit der Bundesrepublik, um ihn herabzusetzen. Er wehrte sich nicht dagegen, doch bekannte er, „Herkunft und üble Nachrede“ hätten ihm einen „Stachel eingepflanzt“.[3] Noch im Bundestagswahlkampf 1965, in dem Brandt für die SPD gegen den CDU-Kandidaten Ludwig Erhard antrat, wurde dieser Umstand thematisiert. Schule und Ausbildung[Bearbeiten | Quelltext bearbeiten] Brandt besuchte die St.-Lorenz-Knaben-Mittelschule, ab 1927 die v. Großheim’sche Realschule und wechselte 1928 zum Johanneum zu Lübeck,[1] an dem er 1932 sein Abitur ablegte. Im Antrag auf Zulassung zum Abitur nannte er Journalist als Berufswunsch. Die SPD stellte Brandt zunächst ein Parteistipendium für ein Hochschulstudium in Aussicht, bis er sich von ihr im Streit getrennt hatte. Stattdessen begann er im Mai 1932 ein Volontariat bei der Schiffsmaklerfirma, Reederei und Spedition F. H. Bertling KG in Lübeck. Politische Betätigung in der Weimarer Republik[Bearbeiten | Quelltext bearbeiten] Nachempfundenes Arbeitszimmer des jungen Willy Brandt (damals Herbert Frahm) im Willy-Brandt-Haus Lübeck Brandts Interesse für Politik ist auf seinen Stiefgroßvater Ludwig Frahm zurückzuführen. Frahm gehörte der SPD an, war zeitweise"
1,2,Willy Brandt,0.316,636,"er im Mai 1932 ein Volontariat bei der Schiffsmaklerfirma, Reederei und Spedition F. H. Bertling KG in Lübeck. Politische Betätigung in der Weimarer Republik[Bearbeiten | Quelltext bearbeiten] Nachempfundenes Arbeitszimmer des jungen Willy Brandt (damals Herbert Frahm) im Willy-Brandt-Haus Lübeck Brandts Interesse für Politik ist auf seinen Stiefgroßvater Ludwig Frahm zurückzuführen. Frahm gehörte der SPD an, war zeitweise Vertrauensmann seiner Partei im Lübecker Stadtbezirk Holstentor-Süd und kandidierte 1926 und 1929 auf der SPD-Liste für die Lübecker Bürgerschaft.[8] Brandt wurde 1925 Mitglied der Kinderfreunde, einer Kindergruppe der Falken, ab April 1929 der Sozialistischen Arbeiter-Jugend (SAJ), in der er als Mitglied der Lübecker Gruppe Karl Marx mit Unterstützung Julius Lebers einen radikalen Kurs vertrat. 1931 wurde Brandt Bezirksvorsitzender der SAJ für die Region Lübeck-Mecklenburg. In diesem Umfeld betätigte sich Brandt seit 1927 regelmäßig publizistisch. Der Lübecker Volksbote, die örtliche SPD-Zeitung, die von Julius Leber redigiert wurde, druckte im Februar 1927 einen Aufsatz Brandts mit zwei Zeichnungen über eine Tageswanderung des Schülers mit Freunden zur Travequelle ab. Ab 1928 veröffentlichte Brandt Texte zu politischen Themen. Leber unterstützte Brandt und förderte zugleich sein politisches Engagement. Brandt erklärte später, Leber habe ihn in diesen Jahren entscheidend beeinflusst. Unter seiner Journalistentätigkeit litten die schulischen Leistungen. Ein Lehrer seiner Schule riet seiner Mutter im Jahr 1930: „Halten Sie Ihren Sohn von der Politik fern. Der Junge hat gute Anlagen. Aber die Politik wird ihn ruinieren“.[9] 1930 trat Brandt der SPD bei. Ein Jahr später, im Oktober 1931 brach er mit Leber und der SPD und warf der Partei – enttäuscht von ihrer Tolerierungspolitik gegenüber den Maßnahmen der konservativen 

In [14]:
biography = df.iloc[0,4] + df.iloc[1,4] + df.iloc[2,4] + df.iloc[3,4] + df.iloc[4,4]
#print(system_prompt)
print(text_to_dqr("gpt-4.1", system_prompt, biography))

(4, 'Abitur 1932, Studium ohne Abschluss', 3, 5212, 14, 0)


In [15]:
biography

'zu dem fünfjährigen Herbert auf. Er nahm ihn in seinen Haushalt auf und behielt ihn während der 1920er Jahre in seiner Obhut, auch nachdem er nach dem Tod Wilhelmines eine neue Ehe mit Dorothea Stahlmann, der von Brandt wenig geliebten „Tante Dora“, eingegangen war. Herbert Frahm nannte seinen Stiefgroßvater „Papa“. In seinem Abiturzeugnis wurde Ludwig Frahm als Vater genannt.[6] Im September 1927 heiratete Herberts Mutter den Polier Emil Kuhlmann, im Februar 1928 kam Herberts Halbbruder Günther Kuhlmann zur Welt. Seit dieser Zeit sah Herbert seine Mutter „nur noch sporadisch“.[4] Rückblickend bezeichnete Brandt seine Jugend als „unbehaust“[3] und beschrieb seine Familienverhältnisse als chaotisch.[7] Brandts uneheliche Geburt, die von Zeitgenossen vielfach als Makel angesehen wurde, benutzten politische Gegner noch in der Zeit der Bundesrepublik, um ihn herabzusetzen. Er wehrte sich nicht dagegen, doch bekannte er, „Herkunft und üble Nachrede“ hätten ihm einen „Stachel eingepflanzt“.

In [14]:
# Schnelle Ausführung:
person = next(item for item in data if item["ID"] == "#017")
example_sections = extract_content_sections(person)
chunked_sections = chunk_content_sections(example_sections)
print(f"Original: {len(example_sections)} Sections")
print(f"Nach Chunking: {len(chunked_sections)} Sections")
section_embeddings = embed_sections(chunked_sections)
top_sections = find_top_k_sections(query_embedding, section_embeddings, chunked_sections, top_k=100)

# In DataFrame umwandeln für strukturierte Darstellung
df = pd.DataFrame([{
    "Rang": i+1,
    "Politiker": result['section']['politician_name'],
    "Score": f"{result['similarity']:.3f}",
    "token_count": count_tokens(result['section']['section_content']),
    "Inhalt": result['section']['section_content']
} for i, result in enumerate(top_sections)])

# Lange Spalteninhalte anzeigen
pd.set_option('display.max_colwidth', None)

# Anzeige
display(df)

biography = df.iloc[0,4] + df.iloc[1,4] + df.iloc[2,4] + df.iloc[3,4] + df.iloc[4,4] + df.iloc[5,4]
#print(system_prompt)
print(text_to_dqr("gpt-4.1", system_prompt, biography))

Gesamtanzahl Abschnitte: 5
Original: 5 Sections
Nach Chunking: 5 Sections


,Rang,Politiker,Score,token_count,Inhalt
0,1,Hannelore Rönsch,0.433,220,"Rönschs Vater war Landwirt und Winzer. Er starb 1946 in russischer Kriegsgefangenschaft.[3] Nach der Volksschule machte Rönsch die Mittlere Reife und besuchte bis 1960 die Höhere Handelsschule. Als Vertrauensschülerin und Klassensprecherin nahm sie an politischen Seminaren der heutigen Landeszentrale für Politische Bildung teil. Von 1960 bis 1962 war sie beim Bundeskriminalamt tätig[4] und von 1962 bis 1983 als Mieterberaterin bei der gemeinnützigen Wohnungsbaugesellschaft Nassauische Heimstätte angestellt.[4][2]\nRönsch ist verwitwet[5], seit dem 31. Oktober 2019 wieder verheiratet[6] und hat eine Tochter,[1] sowie einen Enkel und eine Enkelin.[7]"
1,2,Hannelore Rönsch,0.288,537,"Nach der Bundestagswahl 1990 wurde sie am 18. Januar 1991 als Bundesministerin für Familie und Senioren in die von Bundeskanzler Helmut Kohl geführte Bundesregierung (Kabinett Kohl IV) berufen.[16][1]\nMit der Wiedervereinigung war gemäß Artikel 31 des Einigungsvertrages eine gesetzliche Neuregelung des Schwangerschaftsabbruchs notwendig geworden.[17] Für Rönsch stand dabei der Schutz des ungeborenen Lebens im Vordergrund. Die Fristenlösung, wie sie seit 1972 in der DDR galt, lehnte sie ab.[18] Seit 1993 sind deutschlandweit Abtreibungen in den ersten drei Schwangerschaftsmonaten rechtswidrig, bleiben aber straffrei, wenn sich die Frau mindestens drei Tage vor dem Eingriff beraten lässt.[19]\nRönsch setzte sie sich für die Unterstützung von Familien ein. Kindergeld und Kinderfreibetrag wurden erhöht, der Erziehungsurlaub auf drei Jahre und das Erziehungsgeld auf zwei Jahre verlängert.[20] Ein zentrales Anliegen war die Förderung einkommensschwacher und kinderreicher Familien durch einen verbesserten Familienlastenausgleich.\nAls Ministerin für Familie und Senioren übernahm Rönsch nach der Wiedervereinigung auch die Verantwortung für die 1.400 sanierungsbedürftigen Pflege- und Behinderteneinrichtungen in den neuen Bundesländern[21] und gründete die private Stiftung „Daheim im Heim“ mit einem Stiftungskapital von 500.000 DM. Mit Hilfe von Spenden von Persönlichkeiten aus Kunst, Kultur, Wirtschaft und Politik konnten rund 8 Mio. DM in Pflege- und Behinderteneinrichtungen in den neuen Bundesländern investiert werden.[7]\nNach der Bundestagswahl 1994 schied sie am 17. November 1994 aus dem Kabinett aus[22], das von ihr geführte Ministerium in dem neuen Bundesministerium für Familie, Senioren, Frauen und Jugend aufging.[23]\nAls wichtigste Leistung ihrer Ministerzeit sieht Rönsch, die Wiedervereinigung mitgestaltet zu haben.[7][3]"
2,3,Hannelore Rönsch,0.265,330,"Hannelore Rönsch: Leitsätze zum besseren Schutz ungeborenen Lebens. In: Eichholz Brief, Heft 3 (1991), S. 3–5. Hannelore Rönsch: Die Zukunft älterer Menschen in Europa. Hrsg. v. Europabüro der CDU-CSU-Bundestagsfraktion, Reihe „Europa als Auftrag“, Bonn, Bd. 10, 1992. Statistisches Bundesamt und Hannelore Rönsch: Freizeitverhalten Älterer Menschen. In: Im Blickpunkt: Ältere Menschen, Verlag Metzler-Poeschel, Stuttgart, 1992. Hannelore Rönsch: Der Unterhaltsvorschuß. Eine Hilfe Für Alleinerziehende. Hrsg. vom Bundesministerium für Familie und Senioren, Bonn 1993. Hannelore Rönsch: Familienpolitik und Lebenswirklichkeit – Herausforderungen und Perspektiven. In: 40 Jahre Familienpolitik in der Bundesrepublik Deutschland. Rückblick/Ausblick. Festschrift. Hrsg. vom Bundesministerium für Familie und Senioren. Neuwied u. a. 1993. Hannelore Rönsch: Veränderungen im Netzwerk Familie. In: Rita Süssmuth (Hrsg.): Mut zur Macht in Frauenhand. Herford 2002."
3,4,Hannelore Rönsch,0.248,171,"Nach ihrem Ausscheiden aus der Politik blieb Rönsch Vorsitzende des Kuratoriums der Stiftung „Daheim im Heim“, das sie seit 1993 leitet.[24] 2017 überführte sie die Stiftung in die Trägerschaft des Deutschen Roten Kreuzes.[25]\nVon November 2001 bis November 2014[26] war sie Präsidentin des DRK-Landesverbandes Hessen.[1][9] Das Deutsche Rote Kreuz der Bu

IndexError: index 5 is out of bounds for axis 0 with size 5

In [15]:
print(text_to_dqr("gpt-4.1", system_prompt, biography))

(3, 'Mittlere Reife, Höhere Handelsschule', 3, 3583, 15, 2432)
